This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [1]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory, CombinedMemory, ChatMessageHistory
from langchain.chains import ConversationChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain
from typing import Any, Dict, Optional, Tuple

import re
import requests
import os

os.environ["OPENAI_API_KEY"] = "your api key"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [2]:
model_name = "gpt-3.5-turbo"
temperature = 0.0
llm = OpenAI(model_name=model_name, temperature=temperature)

/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


### Synthetic Data Generation

In [3]:
data_gen_template = '''

Generate {num_listings} real estate listing 

The real estate listing should include a detailed description about the property explaning what makes it special similar to What's special section on Zillow.

Also, make sure that the description includes number of beds, number of baths, living area in sqft, built year, lot area in sqft, asking price, 
number floors, parkings, features types, appliance types, school ratings. However, these should not be in bullet form but in sentence form as a part of description

Provide the output as python list containing {num_listings} elements where each element is one listing.

'''

data_gen_prompt = PromptTemplate.from_template(data_gen_template)

synth_data = llm(data_gen_prompt.format(num_listings=2))

# synth_data = re.findall(r'"(.*)"',synth_data)

# synth_df = pd.DataFrame({'Listing':synth_data})

# synth_df.to_csv('genai_listings.csv',index = False)

print(llm(data_gen_prompt.format(num_listings=2)))

[
    "Welcome to this charming 3 bedroom, 2 bathroom home located in the heart of a family-friendly neighborhood. This home boasts a spacious living area of 1,800 sqft, perfect for entertaining guests or relaxing with loved ones. Built in 2005, this property sits on a generous lot size of 6,000 sqft, providing ample outdoor space for gardening or outdoor activities. The asking price for this lovely home is $350,000. With 2 floors, this home offers plenty of space for a growing family. The property also includes a 2-car garage for convenient parking. Features of this home include a cozy fireplace, hardwood floors, and a beautifully landscaped backyard. The kitchen comes equipped with stainless steel appliances, perfect for cooking delicious meals. This home is zoned for top-rated schools in the area, making it an ideal choice for families.",
    
    "Step into luxury with this stunning 4 bedroom, 3.5 bathroom estate located in a prestigious gated community. This home offers a generous

In [4]:
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.create_documents(synth_data)

embeddings = OpenAIEmbeddings()

db = Chroma.from_documents(split_docs, embeddings)